# Hoja de trabajo 2

Link del repositorio: https://github.com/faguilarleal/HDT2_deep  


Integrantes:  
- Franci Aguilar 22243
- César López 22404


## Ejercicio 1. Experimentación práctica
#### Task 1 - Preparación del conjunto de datos
Cargue el conjunto de datos de Iris utilizando bibliotecas como sklearn.datasets. Luego, divida el conjunto de datos en conjuntos de entrenamiento y validación.


In [3]:
import numpy as np
import copy
import matplotlib.pyplot as plt
import scipy
from PIL import Image
import os
from collections import defaultdict
import torch
from torch import nn

In [5]:
# Importar librerías necesarias
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
import pandas as pd

# Cargar el conjunto de datos Iris
iris = load_iris()

# Convertir a DataFrame para verlo más ordenado
df = pd.DataFrame(data=iris.data, columns=iris.feature_names)
df['target'] = iris.target

print("Primeras filas del dataset:")
print(df.head())

X = iris.data
y = iris.target

X_train, X_val, y_train, y_val = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

print("\nTamaños de los conjuntos:")
print("Entrenamiento:", X_train.shape, y_train.shape)
print("Validación:", X_val.shape, y_val.shape)


Primeras filas del dataset:
   sepal length (cm)  sepal width (cm)  petal length (cm)  petal width (cm)  \
0                5.1               3.5                1.4               0.2   
1                4.9               3.0                1.4               0.2   
2                4.7               3.2                1.3               0.2   
3                4.6               3.1                1.5               0.2   
4                5.0               3.6                1.4               0.2   

   target  
0       0  
1       0  
2       0  
3       0  
4       0  

Tamaños de los conjuntos:
Entrenamiento: (120, 4) (120,)
Validación: (30, 4) (30,)


#### Task 2 - Arquitectura modelo
Cree una red neuronal feedforward simple utilizando nn.Module de PyTorch. Luego, defina capa de entrada, capas ocultas y capa de salida. Después, elija las funciones de activación y el número de neuronas por capa.

In [7]:
import torch.nn.functional as F

class IrisNet(nn.Module):
    def __init__(self):
        super(IrisNet, self).__init__()
        # Capa de entrada -> 4 neuronas 
        # Primera capa oculta -> 16 neuronas
        # Segunda capa oculta -> 8 neuronas
        # Capa de salida -> 3 neuronas (porque iris tiene 3 clases)
        self.fc1 = nn.Linear(4, 16)   # entrada -> capa oculta 1
        self.fc2 = nn.Linear(16, 8)   # capa oculta 1 -> capa oculta 2
        self.fc3 = nn.Linear(8, 3)    # capa oculta 2 -> salida

    def forward(self, x):
        # Funciones de activación
        x = F.relu(self.fc1(x))  
        x = F.relu(self.fc2(x))  
        x = self.fc3(x)         
        return x

# Instanciar el modelo
model = IrisNet()
print(model)


IrisNet(
  (fc1): Linear(in_features=4, out_features=16, bias=True)
  (fc2): Linear(in_features=16, out_features=8, bias=True)
  (fc3): Linear(in_features=8, out_features=3, bias=True)
)


#### Task 3 - Funciones de Pérdida
Utilice diferentes funciones de pérdida comunes como Cross-Entropy Loss y MSE para clasificación. Entrene el modelo con diferentes funciones de pérdida y registre las pérdidas de entrenamiento y test. Debe utilizar al menos 3 diferentes funciones. Es decir, procure que su código sea capaz de parametrizar el uso de diferentes funciones de pérdida.

In [ ]:
import torch.optim as optim

X_train = torch.tensor(X_train, dtype=torch.float32)
y_train = torch.tensor(y_train, dtype=torch.long)
X_val   = torch.tensor(X_val, dtype=torch.float32)
y_val   = torch.tensor(y_val, dtype=torch.long)

#  Funciones de pérdida disponibles
loss_functions = {
    "CrossEntropy": nn.CrossEntropyLoss(),
    "MSE": nn.MSELoss(),  # MSE requiere one-hot en y
    "NLLLoss": nn.NLLLoss()  # requiere log_softmax en salida
}


#  Entrenamiento parametrizable
def train_model(loss_name, epochs=50, lr=0.01):
    model = IrisNet()
    optimizer = optim.Adam(model.parameters(), lr=lr)

    criterion = loss_functions[loss_name]

    # Para NLL necesitamos modificar el forward (añadir log_softmax)
    use_log_softmax = (loss_name == "NLLLoss")

    # Para MSE necesitamos one-hot encoding de las etiquetas
    if loss_name == "MSE":
        y_train_oh = torch.nn.functional.one_hot(y_train, num_classes=3).float()
        y_val_oh   = torch.nn.functional.one_hot(y_val, num_classes=3).float()
    else:
        y_train_oh, y_val_oh = y_train, y_val

    history = {"train_loss": [], "val_loss": []}

    for epoch in range(epochs):
        # --- Forward ---
        outputs = model(X_train)
        if use_log_softmax:
            outputs = torch.log_softmax(outputs, dim=1)

        loss = criterion(outputs, y_train_oh)

        # --- Backprop ---
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        # --- Evaluación en validación ---
        with torch.no_grad():
            val_outputs = model(X_val)
            if use_log_softmax:
                val_outputs = torch.log_softmax(val_outputs, dim=1)
            val_loss = criterion(val_outputs, y_val_oh)

        history["train_loss"].append(loss.item())
        history["val_loss"].append(val_loss.item())

        if (epoch+1) % 10 == 0:
            print(f"[{loss_name}] Epoch {epoch+1}/{epochs} "
                  f"Train Loss: {loss.item():.4f} | Val Loss: {val_loss.item():.4f}")

    return history

# -------------------------------
# 5. Entrenar con las tres funciones
# -------------------------------
hist_ce  = train_model("CrossEntropy", epochs=50)
hist_mse = train_model("MSE", epochs=50)
hist_nll = train_model("NLLLoss", epochs=50)

C:\Users\Francis\AppData\Local\Temp\ipykernel_13540\3936418881.py:3: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  X_train = torch.tensor(X_train, dtype=torch.float32)
C:\Users\Francis\AppData\Local\Temp\ipykernel_13540\3936418881.py:4: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  y_train = torch.tensor(y_train, dtype=torch.long)
C:\Users\Francis\AppData\Local\Temp\ipykernel_13540\3936418881.py:5: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  X_val   = torch.tensor(X_val, dtype=torch.float32)
C:\Users\Francis\AppData\Local\Temp\ipykernel

[CrossEntropy] Epoch 10/50 Train Loss: 0.9479 | Val Loss: 0.9315
[CrossEntropy] Epoch 20/50 Train Loss: 0.6997 | Val Loss: 0.6743
[CrossEntropy] Epoch 30/50 Train Loss: 0.5057 | Val Loss: 0.4973
[CrossEntropy] Epoch 40/50 Train Loss: 0.4258 | Val Loss: 0.4271
[CrossEntropy] Epoch 50/50 Train Loss: 0.3896 | Val Loss: 0.3947
[MSE] Epoch 10/50 Train Loss: 0.2186 | Val Loss: 0.2074
[MSE] Epoch 20/50 Train Loss: 0.1471 | Val Loss: 0.1383
[MSE] Epoch 30/50 Train Loss: 0.1087 | Val Loss: 0.1099
[MSE] Epoch 40/50 Train Loss: 0.1011 | Val Loss: 0.1030
[MSE] Epoch 50/50 Train Loss: 0.0932 | Val Loss: 0.0945
[NLLLoss] Epoch 10/50 Train Loss: 0.9667 | Val Loss: 0.9414
[NLLLoss] Epoch 20/50 Train Loss: 0.6885 | Val Loss: 0.6628
[NLLLoss] Epoch 30/50 Train Loss: 0.4754 | Val Loss: 0.4628
[NLLLoss] Epoch 40/50 Train Loss: 0.3327 | Val Loss: 0.3283
[NLLLoss] Epoch 50/50 Train Loss: 0.2159 | Val Loss: 0.2181
